# Data Preprocessing / Preprocesamiento de Datos

In [1]:
import pandas as pd

### Data for Polynomic Regression / Datos para Regresión Polinómica

In [2]:
df = pd.read_csv('raw/exportaciones-actividad-saldo-comercial-rangos-exportacion-empresa-exportadora-mensual.csv', parse_dates=['indice_tiempo'])
data = df[['indice_tiempo', 'ventas_gasoil']].copy()
data.columns = ['period', 'sales_quantity']
data.loc[:,'moving_avg'] = data['sales_quantity'].shift(1).rolling(window=6, center=False).mean()
data.loc[:,'deseasonalized'] = data['sales_quantity'] - data['moving_avg']
data.dropna(inplace=True)
train = data[(data['period'] >= pd.to_datetime('2010-01-01')) & (data['period'] < pd.to_datetime('2020-01-01'))].copy()
test = data[data['period'] >= pd.to_datetime('2021-02-01')].copy()
train[['period', 'sales_quantity']].to_csv('poly_reg_train.csv', index=False)
test.to_csv('poly_reg_test.csv', index=False)